Name - Satyam Srivastava  
College IIT Kanpur  
Final Year Undergraduate  
satyamsriv22@iitk.ac.in  
+91 6388443533

In [11]:
!pip install praw --quiet
import logging
logging.getLogger("praw").setLevel(logging.ERROR)


In [28]:
import praw

reddit = praw.Reddit(
    client_id="8RNsm7uy-KzMuZtfwtwtMg",
    client_secret="w4nW6HYfrc3YsZQ0JXZr9WrSEE4wLA",
    user_agent="script:reddit.persona:v1.0 (by u/Satyam)"
)

print("✅ Reddit Login Successful:", reddit.read_only)


✅ Reddit Login Successful: True


In [29]:
def fetch_user_data(username):
    user = reddit.redditor(username)
    comments, posts = [], []

    try:
        for c in user.comments.new(limit=100):
            comments.append((c.body, f"https://reddit.com{c.permalink}"))
        for p in user.submissions.new(limit=50):
            content = p.title + "\n" + p.selftext
            posts.append((content, f"https://reddit.com{p.permalink}"))
    except Exception as e:
        print("Error fetching user data:", e)

    return comments, posts


In [30]:
import re

def analyze_persona(comments, posts):
    persona = {
        "traits": [],
        "interests": [],
        "opinions": [],
        "citations": []
    }

    keywords = ["I believe", "I think", "In my opinion", "I feel", "I love", "My favorite"]

    for text, link in comments + posts:
        lower = text.lower()

        if any(k.lower() in lower for k in keywords):
            if "believe" in lower or "think" in lower:
                persona["opinions"].append((text, link))
            elif "love" in lower or "favorite" in lower:
                persona["interests"].append((text, link))

        if re.search(r"\b(curious|friendly|sarcastic|kind|thoughtful)\b", lower):
            persona["traits"].append((text, link))

    persona["citations"] = list(set([link for _, link in comments + posts][:10]))

    return persona


In [31]:
def save_persona(username, persona):
    filename = f"{username}.txt"

    output = f"USER: {username}\n\n--- PERSONA ---\n\n"

    output += "👤 Personality Traits:\n"
    for t in persona["traits"]:
        output += f"- {t[0]} [Source: {t[1]}]\n"

    output += "\n📚 Interests:\n"
    for i in persona["interests"]:
        output += f"- {i[0]} [Source: {i[1]}]\n"

    output += "\n🎯 Opinions:\n"
    for o in persona["opinions"]:
        output += f"- {o[0]} [Source: {o[1]}]\n"

    output += "\n🔗 Source References:\n"
    for c in persona["citations"]:
        output += f"- {c}\n"

    # ✅ Print it in notebook
    print(output)

    # 💾 Save to file
    with open(filename, "w", encoding="utf-8") as f:
        f.write(output)

    print(f"\n✅ Persona saved to: {filename}")
    return filename


In [32]:
def run_persona_generator():
    url = input("🔗 Enter Reddit profile URL: ").strip()
    username = url.rstrip('/').split('/')[-1]

    print(f"\n⏳ Fetching data for: {username}...\n")
    comments, posts = fetch_user_data(username)

    if not comments and not posts:
        print("⚠️ No data found.")
        return

    persona = analyze_persona(comments, posts)
    filename = save_persona(username, persona)

    return filename

filename = run_persona_generator()


🔗 Enter Reddit profile URL: https://www.reddit.com/user/kojied/

⏳ Fetching data for: kojied...

USER: kojied

--- PERSONA ---

👤 Personality Traits:
- Not in Chelsea but Washington square park has the pigeon whisperer. Bear in mind he’s definitely eccentric and curses a lot so might not be a family friendly guy to talk to.
Also the corner of 46th and 6Av always has insane number of pigeons. Like a lot. [Source: https://reddit.com/r/AskNYC/comments/1hpnjqr/ridiculous_question_anybody_know_where_in_chelsea/m4j0ese/]
- Can you actually “work” in AVP?
I’m curious if people have been able to fully port their workflow into AVP.

I’ve been trying to actually use the Vision Pro to get some work done, but without github and visual studio code, it’s not been the easiest (also obviously no terminal). I could use Mac virtual display, but since I have a M1 MBP the latency is noticeable to the point it’s an adverse experience. I already dropped $4k on this device, renewing my MBP just so I can use 

In [33]:
from google.colab import files
files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>